In [1]:
#necessary package import
#Numpy
import numpy as np
from numpy.linalg import multi_dot
from math import factorial, tanh
#Matplotlib
import matplotlib.pyplot as plt
#Scipy
from scipy.linalg import block_diag
#Thewalrus
from thewalrus import perm
#Strawberryfields
import strawberryfields as sf
from strawberryfields.ops import *
#import mplhep as hep
#Qutip
from qutip import *

In [2]:
#Setting Variables
#Rotation gates
"""
R1 = 1
R2 = np.pi/4
R3 = 1
R4 = np.pi/4
"""
R1 = 0.5719
R2 = -1.9782
R3 = 2.0603
R4 = 0.0644
"""
#Beamsplitters
BT1 = np.pi/4;  BP1 = np.pi/2
BT2 = np.pi/4;  BP2 = np.pi/2
BT3 = np.pi/4;  BP3 = np.pi/2
BT4 = np.pi/4;  BP4 = np.pi/2
BT5 = np.pi/4;  BP5 = np.pi/2
BT6 = np.pi/4;  BP6 = np.pi/2
BT7 = np.pi/4;  BP7 = np.pi/2
BT8 = np.pi/4;  BP8 = np.pi/2
"""
BT1 = 0.7804;  BP1 = 0.8578
BT2 = 0.06406; BP2 = 0.5165
BT3 = 0.473;   BP3 = 0.1176
BT4 = 0.563;   BP4 = 0.1517
BT5 = 0.1323;  BP5 = 0.9946
BT6 = 0.311;   BP6 = 0.3231
BT7 = 0.4348;  BP7 = 0.0798
BT8 = 0.4368;  BP8 = 0.6157


In [3]:
#Rotation gate calculation
Uphase = np.diag([np.exp(R1*1j),np.exp(R2*1j),np.exp(R3*1j),np.exp(R4*1j)])

In [4]:
#Beamsplitter calculation
#Put variables
BSargs = [
    (BT1, BP1),
    (BT2, BP2),
    (BT3, BP3),
    (BT4, BP4),
    (BT5, BP5),
    (BT6, BP6),
    (BT7, BP7),
    (BT8, BP8)
]

t_r_amplitudes = [(np.cos(q), np.exp(p*1j)*np.sin(q)) for q,p in BSargs]

BSunitaries = [np.array([[t, -np.conj(r)], [r, t]]) for t,r in t_r_amplitudes]

UBS1 = block_diag(*BSunitaries[0:2])
UBS2 = block_diag([[1]], BSunitaries[2], [[1]])
UBS3 = block_diag(*BSunitaries[3:5])
UBS4 = block_diag([[1]], BSunitaries[5], [[1]])
UBS5 = block_diag(*BSunitaries[6:8])

In [5]:
U = multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, Uphase])
print(np.round(U,4))

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.457 +0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.11  -0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


In [6]:
# Calculating the probability for each state in measure_states
measure_states = [[x, y, z, t] for x in range(5) for y in range(5) for z in range(5) for t in range(5) if x + y + z + t == 4]

In [7]:
input = [1, 1, 1, 1]

def unitary_mapping(output):
    # The two lines below are the extracted row and column indices.
    list_rows = sum([[i] * output[i] for i in range(len(output))],[])
    list_columns = sum([[i] * input[i] for i in range(len(input))],[])
    Umap = U[:,list_columns][list_rows,:]
    return Umap

def probs_theory(output):
    perm_squared = np.abs(perm(unitary_mapping(output), method="ryser"))**2
    denominator = np.prod([factorial(inp) for inp in input]) * np.prod([factorial(out) for out in output])
    return perm_squared / denominator

In [9]:
#Comparing Theory and Simulation
# Initialize the total probability
total_probability_T = 0  

# measure_states의 각 상태에 대해 확률을 출력
for i, state in enumerate(measure_states):
    # 각 상태의 확률을 출력
    # *state는 state 리스트의 요소들을 개별 인자로 전달합니다.
    print(f"Probability of state(Ideal_Theory)     {state}: {probs_theory(state)*100}%")
    total_probability_T += probs_theory(state)
    
# Print the total probability
print(f"Total probability(Ideal_Theory):     {total_probability_T*100}%")

Probability of state(Ideal_Theory)     [0, 0, 0, 4]: 0.9976245740353498%
Probability of state(Ideal_Theory)     [0, 0, 1, 3]: 4.003119536853783%
Probability of state(Ideal_Theory)     [0, 0, 2, 2]: 2.716416141240894%
Probability of state(Ideal_Theory)     [0, 0, 3, 1]: 0.414559085628145%
Probability of state(Ideal_Theory)     [0, 0, 4, 0]: 6.469120990886585%
Probability of state(Ideal_Theory)     [0, 1, 0, 3]: 0.5511529064587104%
Probability of state(Ideal_Theory)     [0, 1, 1, 2]: 0.9347207459242995%
Probability of state(Ideal_Theory)     [0, 1, 2, 1]: 0.0021064451472127665%
Probability of state(Ideal_Theory)     [0, 1, 3, 0]: 1.0291411855438208%
Probability of state(Ideal_Theory)     [0, 2, 0, 2]: 4.844457534042481%
Probability of state(Ideal_Theory)     [0, 2, 1, 1]: 0.46618659065364365%
Probability of state(Ideal_Theory)     [0, 2, 2, 0]: 2.4765343954832963%
Probability of state(Ideal_Theory)     [0, 3, 0, 1]: 9.27180245089878%
Probability of state(Ideal_Theory)     [0, 3, 1, 0]: 6